In [ ]:
#import necessary packages
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder # to encode lables as integers
from sklearn.metrics import confusion_matrix  # built to derive raw accuracy, sensitivity, and specificity.
from sklearn.metrics import accuracy_score # built to derive accuracy
from skimage import feature  #HOG will come from here
from imutils import build_montages #for visualization
from imutils import paths  #helps to extract image path in dataset
import numpy as np  #help statistics and grab random indices
import cv2   #used to read, process, and display images
import os



In [ ]:
#function to quantify image
#this function is used to extract all the features from images
def quantify_image(image):
    #HOG is structural descripter that will capture and quantify changes in local gradient in the input image.
    features = feature.hog(image,orientations=9, pixels_per_cell =(10,10),cells_per_block=(2,2) ,transform_sqrt=True, block_norm ="L1")
    
    return features


In [3]:
def load_split(path):
	# grab the list of images in the input directory, then initialize
	# the list of data (i.e., images) and class labels
	imagePaths = list(paths.list_images(path))
	data = []
	labels = []
	# loop over the image paths
	for imagePath in imagePaths:
		# extract the class label from the filename
		label = imagePath.split(os.path.sep)[-2]
		# load the input image, convert it to grayscale, and resize
		# it to 200x200 pixels, ignoring aspect ratio
		image = cv2.imread(imagePath)
		image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
		image = cv2.resize(image, (200, 200))
		# threshold the image such that the drawing appears as white
		# on a black background
		image = cv2.threshold(image, 0, 255,
			cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
		# quantify the image
		features = quantify_image(image)
		# update the data and labels lists, respectively
		data.append(features)
		labels.append(label)
	# return the data and labels
	return (np.array(data), np.array(labels))

In [4]:
os.listdir("C:/Users/ysaia/Desktop/2-SEMISTER-2/SRP/sindhura-parkinsons-detection-main/sindhura-parkinsons-detection-main/dataset")


['spiral', 'wave']

In [5]:
path_spiral="C:/Users/ysaia/Desktop/2-SEMISTER-2/SRP/sindhura-parkinsons-detection-main/sindhura-parkinsons-detection-main/dataset/spiral"
path_wave="C:/Users/ysaia/Desktop/2-SEMISTER-2/SRP/sindhura-parkinsons-detection-main/sindhura-parkinsons-detection-main/dataset/wave"

In [6]:
trainingpath=os.path.sep.join([path_spiral,"training"])
testingpath=os.path.sep.join([path_spiral,"testing"])
trainingpath1=os.path.sep.join([path_wave,"training"])
testingpath1=os.path.sep.join([path_wave,"testing"])

In [7]:
print("[INFO] loading spiral data...")
(trainX, trainY) = load_split(trainingpath)
(testX, testY) = load_split(testingpath)
print("[INFO] loading wave data...")
(trainX1, trainY1) = load_split(trainingpath1)
(testX1, testY1) = load_split(testingpath1)

[INFO] loading spiral data...
[INFO] loading wave data...


In [8]:
le=LabelEncoder()
trainY=le.fit_transform(trainY)
testY=le.transform(testY)
trainY1=le.fit_transform(trainY1)
testY1=le.transform(testY1)



In [11]:
trails={}
# loop over the number of trials to run
for i in range(0, 5):
	# train the model
	print("[INFO] training model {} of {}...".format(i + 1,5))
	model = RandomForestClassifier(n_estimators=100)
	model.fit(trainX, trainY)
	# make predictions on the testing data and initialize a dictionary
	# to store our computed metrics
	predictions = model.predict(testX)
	metrics = {}
    

[INFO] training model 1 of 5...
[INFO] training model 2 of 5...
[INFO] training model 3 of 5...
[INFO] training model 4 of 5...
[INFO] training model 5 of 5...


In [12]:
cm = confusion_matrix(testY, predictions)
tn=cm[0][0]
fp=cm[0][1]
fn=cm[1][0]
tp=cm[1][1]
metrics["accuracy"] = (tp + tn) / float(cm.sum())
metrics["sensitivity"] = tp / float(tp + fn)
metrics["specificity"] = tn / float(tn + fp)

for (k, v) in metrics.items():
    l = trails.get(k, [])
    l.append(v)
    trails[k] = l
print("FOR SPIRAL DATASET")
for metric in ("accuracy", "sensitivity", "specificity"):
	values = trails[metric]
	mean = np.mean(values)
	std = np.std(values)

	print(metric)
	print("=" * len(metric))
	print("u={:.4f}, o={:.4f}".format(mean, std))
	print("")


FOR SPIRAL DATASET
accuracy
u=0.8000, o=0.0000

sensitivity
u=0.6667, o=0.0000

specificity
u=0.9333, o=0.0000



In [13]:
testingpaths = list(paths.list_images(testingpath))
idxs = np.arange(0, len(testingpaths))
idxs = np.random.choice(idxs, size=(25,), replace=False)
images = []

for i in idxs:
	image = cv2.imread(testingpaths[i])
	output = image.copy()
	output = cv2.resize(output, (128, 128))
	image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
	image = cv2.resize(image, (200, 200))
	image = cv2.threshold(image, 0, 255,
		cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

	features = quantify_image(image)
	preds = model.predict([features])
	label = le.inverse_transform(preds)[0]

	color =(0,255,0) if label == "healthy" else (0, 0, 255)
	cv2.putText(output, label, (3, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
		color, 2)
	images.append(output)
montage = build_montages(images, (128, 128), (5, 5))[0]

cv2.imshow("Output spiral", montage)
cv2.waitKey(0)


-1

In [14]:

# loop over the number of trials to run
for i in range(0, 5):
	# train the model
	print("[INFO] training model {} of {}...".format(i + 1,5))
	model = RandomForestClassifier(n_estimators=100)
	model.fit(trainX1, trainY1)
	# make predictions on the testing data and initialize a dictionary
	# to store our computed metrics
	predictions1 = model.predict(testX1)
	metrics1 = {}
    

[INFO] training model 1 of 5...
[INFO] training model 2 of 5...
[INFO] training model 3 of 5...
[INFO] training model 4 of 5...
[INFO] training model 5 of 5...


In [15]:
cm1 = confusion_matrix(testY1, predictions1)
tn=cm1[0][0]
fp=cm1[0][1]
fn=cm1[1][0]
tp=cm1[1][1]
metrics1["accuracy"] = (tp + tn) / float(cm1.sum())
metrics1["sensitivity"] = tp / float(tp + fn)
metrics1["specificity"] = tn / float(tn + fp)

for (k, v) in metrics1.items():
    l = trails.get(k, [])
    l.append(v)
    trails[k] = l
print("FOR WAVE DATA SET")
for metric in ("accuracy", "sensitivity", "specificity"):
	values = trails[metric]
	mean = np.mean(values)
	std = np.std(values)

	print(metric)
	print("=" * len(metric))
	print("u={:.4f}, o={:.4f}".format(mean, std))
	print("")


FOR WAVE DATA SET
accuracy
u=0.7833, o=0.0167

sensitivity
u=0.7000, o=0.0333

specificity
u=0.8667, o=0.0667



In [16]:
testingpaths1 = list(paths.list_images(testingpath1))
idxs = np.arange(0, len(testingpaths1))
idxs = np.random.choice(idxs, size=(25,), replace=False)
images = []

for i in idxs:
	image = cv2.imread(testingpaths1[i])
	output = image.copy()
	output = cv2.resize(output, (128, 128))
	image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
	image = cv2.resize(image, (200, 200))
	image = cv2.threshold(image, 0, 255,
		cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

	features = quantify_image(image)
	preds = model.predict([features])
	label = le.inverse_transform(preds)[0]

	color =(0,255,0) if label == "healthy" else (0, 0, 255)
	cv2.putText(output, label, (3, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
		color, 2)
	images.append(output)
montage = build_montages(images, (128, 128), (5, 5))[0]

cv2.imshow("Output", montage)
cv2.waitKey(0)


-1

In [17]:
s_score = accuracy_score(testY,predictions)
print("accuracy of model for spiral dataset",s_score)
w_score = accuracy_score(testY1,predictions1)
print("accuracy of model for wave datase",w_score)

accuracy of model for spiral dataset 0.8
accuracy of model for wave datase 0.7666666666666667
